In [ ]:
%matplotlib inline

from pathlib import Path
import matplotlib.pyplot as plt

import bgcArgoDMQC as bgc

First, you have to set up the module to tell it where to look for data. Note that Argo data stored locally should have the same path structure as the GDAC, i.e., `/Argo/\[dac name\]/\[float WMO\]/profiles/`.

In [ ]:
# define the paths
data_path = Path.cwd() / 'ex-data'
argo_path = data_path / 'Argo'
ncep_path = data_path / 'NCEP'
woa_path  = data_path / 'WOA18'

# tell bgcArgoDMQC where to look
bgc.set_dirs(
    argo_path=argo_path,
    ncep_path=ncep_path,
    woa_path=woa_path
)

Then, you can load in a synthetic profile of the Argo data using `bgc.sprof(...)` - this will create an object that contains all the data from that file and various methods to calculate gain, visualize the data, or export it.

In [ ]:
# our float number
wmo = 4902481
# create a synthetic profile (sprof) object
syn = bgc.sprof(wmo)

Let's calculate the gain for this float using in-air data:

In [ ]:
gains = syn.calc_gains()

and visualize:

In [ ]:
g1 = syn.plot('gain', ref='NCEP')